In [1]:
from azureml.core import Workspace, Experiment
 
ws = Workspace.get(name="quick-starts-ws-133070")
exp = Experiment(workspace=ws, name="udacity-project")
 
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id,
      'Resource group: ' + ws.resource_group, sep = '\n')
 
run = exp.start_logging()

Workspace name: quick-starts-ws-133070
Azure region: southcentralus
Subscription id: 510b94ba-e453-4417-988b-fbdc37b55ca7
Resource group: aml-quickstarts-133070


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
cluster_name = "azure-project"
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target, using it')
except ComputeTargetException:
    print('Creating a new compute target!')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    # create the cluster
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
    
cpu_cluster.wait_for_completion(show_output=True)
 
# use get_status() to get a detailed status for the current cluster.
print(cpu_cluster.get_status().serialize())

Found existing compute target, using it

Running
{'errors': [], 'creationTime': '2021-01-03T05:17:25.692053+00:00', 'createdBy': {'userObjectId': '313adcbd-b320-48a5-9bff-effbbdc56e00', 'userTenantId': '660b3398-b80e-49d2-bc5b-ac1dc93b5254', 'userName': None}, 'modifiedTime': '2021-01-03T05:20:11.494000+00:00', 'state': 'Running', 'vmSize': 'STANDARD_DS2_V2'}


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import choice
import os
 
# Specify parameter sampler
ps = RandomParameterSampling({"--C": uniform(0.01, 10),"--max_iter": choice(100,150,200,250)})
 
# Specify a Policy
policy = BanditPolicy(slack_factor=0.1,evaluation_interval=3)
 
if "training" not in os.listdir():
    os.mkdir("./training")
    
# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=".", compute_target=cpu_cluster, entry_script="train.py")
 
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, 
                                     policy=policy, 
                                     primary_metric_name="Accuracy", 
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                     max_total_runs=10, 
                                     max_concurrent_runs=4, 
                                     estimator=est)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
 
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [5]:
# Get your best run and save the model from that run.
 
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
 
print('Best Run Id: ', best_run.id)
print('Accuracy of Best Run is:', best_run_metrics['Accuracy'])
print('Parameter Values are:',best_run.get_details()['runDefinition']['arguments'])

Best Run Id:  HD_2d8c45e6-2e2b-4675-a3e4-ff2e172c9949_2
Accuracy of Best Run is: 0.9133535660091047
Parameter Values are: ['--C', '6.429755013675407', '--max_iter', '250']


In [6]:
model = best_run.register_model(model_name='hyperdrive-model', 
                                model_path='outputs/model.joblib', 
                                tags={'Method':'Hyperdrive'}, 
                                properties={'Accuracy': best_run_metrics['Accuracy']})

In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory
 
# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
 
url="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds=TabularDatasetFactory.from_delimited_files(path=url)

In [8]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [9]:
#Split data into training and testing
from sklearn.model_selection import train_test_split  
import pandas as pd

x_train,x_test,y_train,y_test= train_test_split(x,y, train_size=0.8, test_size=0.2, random_state=21)

#Concating the x_train & y_train together
training_data=pd.concat((x_train,y_train),axis=1)

In [10]:
from azureml.train.automl import AutoMLConfig
from azureml.core import Dataset, Datastore
                      
# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(experiment_timeout_minutes=30,
                             compute_target=cpu_cluster,
                             task="classification",
                             primary_metric="accuracy",
                             training_data=ds,
                             label_column_name="y",
                             n_cross_validations=4)

In [11]:
# Submit your automl run

automl_run = exp.submit(automl_config, show_output=True)
RunDetails(automl_run).show()

Running on remote.
No run_configuration provided, running on azure-project with default configuration
Running on remote compute: azure-project
Parent Run ID: AutoML_dcdceefb-3f10-44a6-bbaf-8c09f23954a0

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the i

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [12]:
# Retrieve and save your best automl model.

best_run, fitted_model = automl_run.get_output()
print(best_run)

best_run_metrics = best_run.get_metrics()
print('Best Run Id: ', best_run.id)

Run(Experiment: udacity-project,
Id: AutoML_dcdceefb-3f10-44a6-bbaf-8c09f23954a0_24,
Type: azureml.scriptrun,
Status: Completed)
Best Run Id:  AutoML_dcdceefb-3f10-44a6-bbaf-8c09f23954a0_24


In [13]:
print(fitted_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                objective=None,
                                                                                                random_state=None,
                                                                                                reg_alpha=0.5789473684210527,
                                       

In [22]:
from pprint import pprint

def print_model(fitted_model, prefix=""):
    for step in fitted_model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(fitted_model)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['17', '0', '1', '11', '6', '19', '22'],
 'weights': [0.13333333333333333,
             0.2,
             0.13333333333333333,
             0.06666666666666667,
             0.2,
             0.13333333333333333,
             0.13333333333333333]}

17 - sparsenormalizer
{'copy': True, 'norm': 'l1'}

17 - xgboostclassifier
{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 0.5,
 'eta': 0.3,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 6,
 'max_leaves': 0,
 'min_child_weight': 1,
 'missing': nan,
 'n_estimators': 50,
 'n_jobs': 1,
 'n

In [14]:
# Model performace

for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name,"-" , metric)

weighted_accuracy - 0.9589790086061114
f1_score_weighted - 0.9130687489816591
average_precision_score_micro - 0.9818967590971357
f1_score_micro - 0.9171167420803277
f1_score_macro - 0.7716817501267204
average_precision_score_macro - 0.8272205017673206
recall_score_macro - 0.7485743987300189
AUC_macro - 0.9487649689860091
precision_score_weighted - 0.9110511188806671
AUC_weighted - 0.9487649689860091
accuracy - 0.9171167420803277
recall_score_micro - 0.9171167420803277
norm_macro_recall - 0.497148797460038
AUC_micro - 0.9810948738004792
average_precision_score_weighted - 0.9561436103169266
precision_score_macro - 0.8028016307560633
recall_score_weighted - 0.9171167420803277
matthews_correlation - 0.548480377391665
balanced_accuracy - 0.7485743987300189
precision_score_micro - 0.9171167420803277
log_loss - 0.18015918921268873
confusion_matrix - aml://artifactId/ExperimentRun/dcid.AutoML_dcdceefb-3f10-44a6-bbaf-8c09f23954a0_24/confusion_matrix
accuracy_table - aml://artifactId/ExperimentR

In [15]:
# Register the model

automodel = best_run.register_model(model_name='automl_output', 
                                    model_path='./outputs/', 
                                    tags={'Method':'AutoML'},
                                    properties={'Accuracy': best_run_metrics['accuracy']})